In [1]:
import pandas as pd
from os import listdir
import uuid
from datetime import date
import datetime
import numpy as np
import re

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Functions

In [3]:
def load_africa_data():
    africa = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/input_panel_africa.xlsx')
    links = pd.read_excel('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/fact_checks/fact_checks_clean.xlsx')[['Link', 
'id_desinformacion']].drop_duplicates().rename(columns = {'Link':'link_desinformacion'})
    all_data = pd.merge(left = africa, 
            right = links, 
            on = 'id_desinformacion', 
            how = 'left')
    return(all_data)

In [7]:
def load_factchecks():
    factchecks = pd.read_json('../../../../data/1-factchecks/2-clean_factchecks/factchecks_data.json')
    return(factchecks)

In [69]:
def dates_function(x):
    try:
        if pd.isna(x) or  type(x) is str:
            return(None)
        else:
            return(x.strftime('%Y-%m-%d'))
    except:
        return(x)

In [59]:
def load_misinformation_newspapers():
    path = '/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/04-fake_news_dataset/newspapers/Fact-Checkers/'
    data = []
    for i in listdir(path):
        try:
            data.append(pd.read_excel(path + i))
        except:
            pass
        
    misinformation = pd.concat(data)
    misinformation = misinformation[[ 'id_desinformacion', 'fecha_desinformacion', 
               'link_desinformacion',  'label_desinformacion', 
                                    'id_chequeo']].drop_duplicates().rename(columns = {'id_chequeo':'id_factcheck'})
    
    ## include africa
    africa = load_africa_data()
    africa['fecha_desinformacion'] = None
    africa = africa[list(misinformation.columns)]
    
    all_data = pd.concat([misinformation, africa])
    all_data = all_data[~all_data['link_desinformacion'].isna()]
    
    factchecks = load_factchecks()
    
    all_data = pd.merge(left = all_data, 
            right = factchecks, 
            on = 'id_factcheck', 
            how = 'left')
    
    all_data = all_data[~all_data['id_factcheck'].isna()]
    all_data = all_data.drop(['difference_dates'], axis = 1)
    all_data['fecha_desinformacion'] = all_data['fecha_desinformacion'].apply(lambda x: dates_function(x))
    return(all_data)

## Apply Functions

In [70]:
misinformation_data = load_misinformation_newspapers()

In [71]:
misinformation_data.to_excel('../../../../data/2-misinformation/newspapers/0-misinformation/misinformation_newspapers2.xlsx')

In [67]:
misinformation_data

,id_desinformacion,fecha_desinformacion,link_desinformacion,label_desinformacion,id_factcheck,type_factcheck,organizacion,pais,facebook_partnership_date,facebook_partnership,titulo_chequeo,link_chequeo,fecha_chequeo,link_facebook_factcheck,date_factcheck_facebook,dummy_facebook_date,date_factcheck_final
22,942f32bb-6a7d-3291-970d-75afea2c338d,2020-12-07,http://diario1588.com/2020/07/el-dengue-llego-...,fake,5397574c-fd64-3c16-8023-fe398b6ac145,medios_tradicionales,larepublica,peru,2020-06-01,1.0,es falso que china registro el primer caso de ...,https://larepublica.pe/verificador/2020/07/12/...,2020-12-07,None,None,0.0,2020-12-07
23,e83a77f8-6383-3442-8b38-22153f2c4605,2020-05-16,https://notitotal.net/prenden-fuego-a-dos-vene...,fake,f1c88415-f9ee-3d45-bd94-0c4df5192bed,medios_tradicionales,larepublica,peru,2020-06-01,1.0,peru es falso que prendieron fuego a dos ciuda...,https://larepublica.pe/verificador/2020/05/17/...,2020-05-17,None,None,0.0,2020-05-17
24,5c03e43a-bd7a-359b-9bfa-419af1ae1f03,2020-04-23,https://travelnostop.com/sicilia/cronaca/48996...,misleading,35a5dd29-d5d7-323c-8da0-ddf50d9e249a,medios_tradicionales,larepublica,peru,2020-06-01,1.0,coronavirus es enganoso afirmar que pagaran vu...,https://larepublica.pe/verificador/2020/04/29/...,2020-04-29,None,None,0.0,2020-04-29
25,9e8a80b9-16a1-3196-a1b1-dc70e9a085ba,2015-07-07,https://www.hispanidad.com/trazos-publicitario...,fake,81dce861-bc89-326e-9b83-52b12e1b4155,medios_tradicionales,larepublica,peru,2020-06-01,1.0,onu es falso que la exsecretaria christiana fi...,https://larepublica.pe/verificador/2020/06/28/...,2020-06-28,None,None,0.0,2020-06-28
26,585a98b3-0cda-3fd5-902b-7d9667708307,2020-06-21,https://plataforma.quieroauditoriaenergetica.o...,fake,7ec017a5-26c4-39ef-b23a-28b62a9c3630,medios_tradicionales,larepublica,peru,2020-06-01,1.0,tecnologia 5g es falso que sea toxica para los...,https://larepublica.pe/verificador/2020/06/21/...,2020-06-21,None,None,0.0,2020-06-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2143,21b05bc4-4a90-3c8a-9abe-4d687757c4f6,None,https://africacheck.org/wp-content/uploads/201...,fake,49d7af65-82b9-3493-a672-ed225bac8b5d,medios_tradicionales,africa_check,africa,2019-08-15,1.0,flawed data undermines sa claims vaccination c...,https://africacheck.org/fact-checks/reports/fl...,2012-11-21,None,2012-11-21,1.0,2012-11-21
2144,0f7906d0-f6e4-3de6-806c-3f5ce130322b,None,https://web.archive.org/web/20140708212551/htt...,true,5ed4a06f-f695-3b6c-9ebe-af5cffd5b05d,medios_tradicionales,africa_check,africa,2019-08-15,1.0,zille says cape town rich poor divide narrowes...,https://africacheck.org/fact-checks/reports/zi...,2012-12-03,None,None,0.0,2012-12-03
2145,8715caa4-2a02-3987-ab98-0c8e2a934736,None,https://web.archive.org/web/20121202103344/htt...,true,5c1725d5-3f40-3277-9c1b-6bfc27d21bb9,medios_tradicionales,africa_check,africa,2019-08-15,1.0,did gun control cause fall gun crime data back...,https://africacheck.org/fact-checks/reports/di...,2012-12-15,None,None,0.0,2012-12-15
2146,bf32f688-5513-3990-b1ff-4ca22c216b8e,None,https://www.youtube.com/watch?v=sAppABMVmiA,NaN,352681e4-dff5-3dea-b07a-7a5c75875778,medios_tradicionales,africa_check,africa,2019-08-15,1.0,fact checking president kenyatta kenyas small ...,https://africacheck.org/fact-checks/reports/fa...,2020-03-03,None,2020-03-03,1.0,2020-03-03


In [40]:
misinformation_data.to_json('/Users/cblanesg/misinformation_socialmedia/data/2-misinformation/newspapers/0-misinformation/misinformation.json')

In [41]:
pd.read_json('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/newspapers/1-engagement/facebook/posts_engagement.json')

,platformId,platform,date,updated,type,title,caption,description,message,expandedLinks,...,subscriberCount,score,media,statistics,account,history,legacyId,id,imageText,videoLengthMS
0,2.177702e+29,Facebook,2020-01-02 11:23:30,2020-11-19 05:33:30,link,"More knocks, less praise for Buhari on New Yea...",punchng.com,Our Correspondents Some prominent Nigerians an...,https://punchng.com/more-knocks-less-praise-fo...,[{'original': 'https://punchng.com/more-knocks...,...,0,-0.833333,"[{'type': 'photo', 'url': 'https://external.xx...","{'actual': {'likeCount': 3, 'shareCount': 0, '...","{'id': 10119508, 'name': 'Conscience of the Na...","[{'actual': {'likeCount': 3, 'shareCount': 0, ...",81182447505,10119508|740242043130135,None,NaN
1,4.181566e+30,Facebook,2020-01-02 04:01:02,2020-09-29 08:17:33,link,"More knocks, less praise for Buhari on New Yea...",punchng.com,Our Correspondents Some prominent Nigerians an...,None,[{'original': 'https://punchng.com/more-knocks...,...,0,-0.909091,"[{'type': 'photo', 'url': 'https://external.xx...","{'actual': {'likeCount': 1, 'shareCount': 0, '...","{'id': 10124099, 'name': 'The Oak Newspaper', ...","[{'actual': {'likeCount': 1, 'shareCount': 0, ...",0,10124099|1444084329076047,None,NaN
2,1.643974e+31,Facebook,2020-03-05 17:20:43,2020-10-22 01:11:46,link,ANALYSIS: Think twice about the ‘Spekboom chal...,africacheck.org,Consider other indigenous plants that have evo...,Est-ce qu'on a quelque chose similaire ici!?? ...,[{'original': 'https://www.goodthingsguy.com/e...,...,0,-6.750000,"[{'type': 'photo', 'url': 'https://external.xx...","{'actual': {'likeCount': 2, 'shareCount': 1, '...","{'id': 10216311, 'name': 'SOLS VIVANTS QUÉBEC ...","[{'actual': {'likeCount': 2, 'shareCount': 1, ...",82339310495,10216311|2563084690576718,None,NaN
3,1.119044e+30,Facebook,2020-01-23 05:27:09,2020-10-13 00:54:04,link,SA wonder plant removes more CO2 than Amazon,getaway.co.za,"Spekboom, our own indigenous wonder plant, can...",Morning ☺️ not gatvol Looking for SPEKBOOM sli...,[{'original': 'https://www.getaway.co.za/trave...,...,0,1.000000,"[{'type': 'photo', 'url': 'https://external.xx...","{'actual': {'likeCount': 19, 'shareCount': 12,...","{'id': 10116229, 'name': 'GATVOL BLUFF', 'prof...","[{'actual': {'likeCount': 19, 'shareCount': 12...",0,10116229|2979845362025998,None,NaN
4,9.201504e+30,Facebook,2019-12-09 19:31:26,2020-10-14 00:42:31,link,SA wonder plant removes more CO2 than Amazon,getaway.co.za,"Spekboom, our own indigenous wonder plant, can...",Not sure this is quite what it's being hyped a...,[{'original': 'https://www.getaway.co.za/trave...,...,0,-1.718750,"[{'type': 'photo', 'url': 'https://external.xx...","{'actual': {'likeCount': 18, 'shareCount': 9, ...","{'id': 6876106, 'name': 'Regenerative Consciou...","[{'actual': {'likeCount': 18, 'shareCount': 9,...",0,6876106|1193350730870250,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6234,2.160444e+29,Facebook,2013-07-04 08:10:25,2020-11-30 21:01:27,link,Join the ANC,anc.org.za,Official web site of the African National Cong...,It is time to make a good choice.................,[{'original': 'http://www.anc.org.za/show.php?...,...,0,-7.666667,None,"{'actual': {'likeCount': 1, 'shareCount': 0, '...","{'id': 8525840, 'name': 'ANC YOUTH LIVE', 'pro...","[{'actual': {'likeCount': 1, 'shareCount': 0, ...",75707770129,8525840|513874565344315,None,NaN
6235,1.704000e+29,Facebook,2013-06-15 21:57:02,2020-10-27 04:25:19,link,ANC statement on the execution of Solomon Mahl...,anc.org.za,Official web site of the African National Cong...,SHARING HIDDEN HISTORIES: Saturday 15th June 2...,[{'original': 'http://www.anc.org.za/show.php?...,...,0,-12.333333,None,"{'actual': {'likeCount': 3, 'shareCount': 0, '...","{'id': 7652030, 'name': 'GREATEST AFRICANS IN ...","[{'actual': {'likeCount': 3, 'shareCount': 0, ...",0,7652030|610625948949427,None,NaN
6236,3.586640e+29,Facebook,2013-06-11 07:06:30,2020-11-25 02:15:37,status,None

In [42]:
pd.read_json('/Users/cblanesg/cam.blanes Dropbox/Camila Blanes/Bolivia_Project/data/04-fakenews-Repository/06-virality/africa/newspapers/1-engagement/facebook/disaggregated_engagements.json')

,timestep,date,likeCount,shareCount,commentCount,loveCount,wowCount,hahaCount,sadCount,angryCount,thankfulCount,careCount,n_users,id_desinformacion,postUrl,id,date_publication_init
0,74,2020-11-19 05:33:30,3,0,0,0,0,0,0,0,0,0,1,f00c11cf-cb53-39a3-9c87-deb2026b252b,https://www.facebook.com/groups/21777018537732...,10119508|740242043130135,2020-01-02 11:23:30
1,74,2020-09-29 08:17:33,1,0,0,0,0,0,0,0,0,0,1,f00c11cf-cb53-39a3-9c87-deb2026b252b,https://www.facebook.com/groups/41815656833550...,10124099|1444084329076047,2020-01-02 04:01:02
2,74,2020-10-22 01:11:46,2,1,1,0,0,0,0,0,0,0,1,d9c74c28-0cd7-3dab-84ff-75f479f0c4bf,https://www.facebook.com/groups/16439736124878...,10216311|2563084690576718,2020-03-05 17:20:43
3,73,2020-03-24 22:19:53,2,1,1,0,0,0,0,0,0,0,1,d9c74c28-0cd7-3dab-84ff-75f479f0c4bf,https://www.facebook.com/groups/16439736124878...,10216311|2563084690576718,2020-03-05 17:20:43
4,72,2020-03-24 00:13:33,2,1,1,0,0,0,0,0,0,0,1,d9c74c28-0cd7-3dab-84ff-75f479f0c4bf,https://www.facebook.com/groups/16439736124878...,10216311|2563084690576718,2020-03-05 17:20:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154873,74,2020-12-02 20:40:07,0,0,1,0,0,0,0,0,0,0,1,4fe24770-9283-3f8e-8f96-51ee51da5ab2,https://www.facebook.com/groups/11158090220792...,10713343|647049015327777,2013-09-23 10:58:05
154874,74,2020-11-30 21:01:27,1,0,2,0,0,0,0,0,0,0,1,4fe24770-9283-3f8e-8f96-51ee51da5ab2,https://www.facebook.com/groups/21604435512733...,8525840|513874565344315,2013-07-04 08:10:25
154875,74,2020-10-27 04:25:19,3,0,0,0,0,0,0,0,0,0,1,4fe24770-9283-3f8e-8f96-51ee51da5ab2,https://www.facebook.com/groups/17040003963868...,7652030|610625948949427,2013-06-15 21:57:02
154876,74,2020-11-25 02:15:37,0,0,0,0,0,0,0,0,0,0,1,4fe24770-9283-3f8e-8f96-51ee51da5ab2,https://www.facebook.com/groups/35866397085115...,8520179|566644133386469,2013-06-11 07:06:30
